In [3]:
import re
import glob
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

This is code to get a trivial model running.

No train/validation split here, using all data to train the model.

Also make a prediction and verify the exact meaning of the loss, and mean absolute error.

In [11]:
# Load simulated data: load complete training and validation sets.
np.random.seed(11)

# Training set should be one concatenated shuffled list of simulated examples
# Validation set similarly, but also keep information about simulation parameters
# to evaluate performance in specific simulation conditions.
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]
y = np.concatenate([np.loadtxt(bounds_file) for bounds_file in bounds_files])
x = np.concatenate([np.loadtxt(intensities_file) for intensities_file in intensities_files])

In [24]:
def overlap_metric(y_true, y_predicted):
    y_pred_round = tf.round(y_predicted)
    max_start = tf.math.maximum(y_true[0], y_pred_round[0])
    min_end = tf.math.minimum(y_true[1], y_pred_round[1])
    overlap = tf.math.maximum(0.0, min_end - max_start)
    longest_bounds = tf.math.maximum(y_true[1] - y_true[0], y_pred_round[1] - y_pred_round[0])
    return overlap / longest_bounds

In [25]:
model = tf.keras.Sequential([
        layers.Dense(40, activation='relu'),
        layers.Dense(2, activation='linear')
    ])
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='mse', 
              metrics=['mae', overlap_metric])

In [26]:
history = model.fit(x, y, epochs=10, validation_split=0.33)

Train on 60299 samples, validate on 29701 samples
Epoch 1/10
60299/60299 [==============================] - 5s 80us/step - loss: 57.7866 - mean_absolute_error: 4.6690 - overlap_metric: nan - val_loss: 11.0044 - val_mean_absolute_error: 2.5516 - val_overlap_metric: nan
Epoch 2/10
60299/60299 [==============================] - 5s 76us/step - loss: 9.1662 - mean_absolute_error: 2.3567 - overlap_metric: nan - val_loss: 11.7012 - val_mean_absolute_error: 2.7890 - val_overlap_metric: nan
Epoch 3/10
60299/60299 [==============================] - 6s 92us/step - loss: 7.7180 - mean_absolute_error: 2.1513 - overlap_metric: nan - val_loss: 6.2147 - val_mean_absolute_error: 1.9268 - val_overlap_metric: nan
Epoch 4/10
60299/60299 [==============================] - 4s 66us/step - loss: 6.1214 - mean_absolute_error: 1.9104 - overlap_metric: nan - val_loss: 5.2350 - val_mean_absolute_error: 1.7475 - val_overlap_metric: nan
Epoch 5/10
60299/60299 [==============================] - 4s 61us/step - loss: 